In [1]:
import pandas as pd
import csv

import pyspark
import findspark
findspark.init()
import re
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql import SQLContext
spark = SparkSession.builder.appName('Homework3').getOrCreate()

In [2]:
amazon_data = pd.read_csv('C:/Users/Sachin Chavan/Downloads/Amazon_Responded_Oct05.csv') #importing the dataset
amazon_data.head() #viewing the dataset

,id_str,tweet_created_at,user_screen_name,user_id_str,user_statuses_count,user_favourites_count,user_protected,user_listed_count,user_following,user_description,...,tweet_language,text_,favorite_count,favorited,in_reply_to_screen_name,in_reply_to_status_id_str,in_reply_to_user_id_str,retweet_count,retweeted,text
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,'793270689780203520',Tue Nov 01 01:57:25 +0000 2016,SeanEPanjab,143515471.0,51287.0,4079.0,False,74.0,False,Content marketer; Polyglot; Beard aficionado; ...,...,en,@AmazonHelp Can you please DM me? A product I ...,0.0,False,AmazonHelp,NaN,85741735.0,0.0,False,NaN
2,'793281386912354304',Tue Nov 01 02:39:55 +0000 2016,AmazonHelp,85741735.0,2225450.0,11366.0,False,796.0,False,We answer Amazon support questions 7 days a we...,...,en,"@SeanEPanjab I'm sorry, we're unable to DM you...",0.0,False,SeanEPanjab,7.930000e+17,143515471.0,0.0,False,NaN
3,'793501578766319616',Tue Nov 01 17:14:53 +0000 2016,SeanEPanjab,143515471.0,51287.0,4079.0,False,74.0,False,Content marketer; Polyglot; Beard aficionado; ...,...,en,@AmazonHelp It was purchased on https://t.co/g...,0.0,False,AmazonHelp,7.930000e+17,85741735.0,0.0,False,@AmazonHelp It was purchased on https://t.co/g...
4,'793501657346682880',Tue Nov 01 17:15:12 +0000 2016,SeanEPanjab,143515471.0,51287.0,4079.0,False,74.0,False,Content marketer; Polyglot; Beard aficionado; ...,...,en,"@AmazonHelp I am following you now, if it help...",0.0,False,AmazonHelp,7.930000e+17,85741735.0,0.0,False,NaN


In [3]:
print(amazon_data.shape) #checking the number of rows and columns

(462029, 25)


In [4]:
print(amazon_data.columns) #checking column names

Index(['id_str', 'tweet_created_at', 'user_screen_name', 'user_id_str',
       'user_statuses_count', 'user_favourites_count', 'user_protected',
       'user_listed_count', 'user_following', 'user_description',
       'user_location', 'user_verified', 'user_followers_count',
       'user_friends_count', 'user_created_at', 'tweet_language', 'text_',
       'favorite_count', 'favorited', 'in_reply_to_screen_name',
       'in_reply_to_status_id_str', 'in_reply_to_user_id_str', 'retweet_count',
       'retweeted', 'text'],
      dtype='object')


In [5]:
print(amazon_data.info()) #gathering information regarding our dataset

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 462029 entries, 0 to 462028
Data columns (total 25 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   id_str                     378134 non-null  object 
 1   tweet_created_at           378134 non-null  object 
 2   user_screen_name           378134 non-null  object 
 3   user_id_str                378134 non-null  float64
 4   user_statuses_count        378134 non-null  float64
 5   user_favourites_count      378134 non-null  float64
 6   user_protected             378134 non-null  object 
 7   user_listed_count          378134 non-null  float64
 8   user_following             378134 non-null  object 
 9   user_description           331689 non-null  object 
 10  user_location              155919 non-null  object 
 11  user_verified              378134 non-null  object 
 12  user_followers_count       378134 non-null  float64
 13  user_friends_count         37

In [6]:
amazon_data['Month'] = pd.DatetimeIndex(amazon_data['tweet_created_at']).month #using datetime function of pandas to extarct month
amazon_data['Month'] = pd.to_datetime(amazon_data['Month'], format='%m').dt.month_name().str.slice(stop=3) #converting it to string format to match the question 
amazon_data['Day'] = pd.DatetimeIndex(amazon_data['tweet_created_at']).day #extracting day using pandas datetime function
amazon_data['Year'] = pd.DatetimeIndex(amazon_data['tweet_created_at']).year #extracting day using pandas datetime function

In [7]:
amazon_data['tweet_created_date'] = amazon_data['Month'] + " " + amazon_data['Day'].astype(str) + " " + amazon_data['Year'].astype(str) #concatinating the two columns
amazon_data #displaying the dataset

,id_str,tweet_created_at,user_screen_name,user_id_str,user_statuses_count,user_favourites_count,user_protected,user_listed_count,user_following,user_description,...,in_reply_to_screen_name,in_reply_to_status_id_str,in_reply_to_user_id_str,retweet_count,retweeted,text,Month,Day,Year,tweet_created_date
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,'793270689780203520',Tue Nov 01 01:57:25 +0000 2016,SeanEPanjab,143515471.0,51287.0,4079.0,False,74.0,False,Content marketer; Polyglot; Beard aficionado; ...,...,AmazonHelp,NaN,85741735.0,0.0,False,NaN,Nov,1.0,2016.0,Nov 1.0 2016.0
2,'793281386912354304',Tue Nov 01 02:39:55 +0000 2016,AmazonHelp,85741735.0,2225450.0,11366.0,False,796.0,False,We answer Amazon support questions 7 days a we...,...,SeanEPanjab,7.930000e+17,143515471.0,0.0,False,NaN,Nov,1.0,2016.0,Nov 1.0 2016.0
3,'793501578766319616',Tue Nov 01 17:14:53 +0000 2016,SeanEPanjab,143515471.0,51287.0,4079.0,False,74.0,False,Content marketer; Polyglot; Beard aficionado; ...,...,AmazonHelp,7.930000e+17,85741735.0,0.0,False,@AmazonHelp It was purchased on https://t.co/g...,Nov,1.0,2016.0,Nov 1.0 2016.0
4,'793501657346682880',Tue Nov 01 17:15:12 +0000 2016,SeanEPanjab,143515471.0,51287.0,4079.0,False,74.0,False,Content marketer; Polyglot; Beard aficionado; ...,...,AmazonHelp,7.930000e+17,85741735.0,0.0,False,NaN,Nov,1.0,2016.0,Nov 1.0 2016.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
462024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
462025,'892532821893533697',Tue Aug 01 23:49:41 +0000 2017,nat_says_,60186579.0,14857.0,229.0,False,5.0,False,nothing really going on here. just talking to ...,...,AmazonHelp,NaN,85741735.0,0.0,False,NaN,Aug,1.0,2017.0,Aug 1.0 2017.0
462026,'892533275968798720',Tue Aug 01 23:51:30 +0000 2017,AmazonHelp,85741735.0,2215792.0,11277.0,False,796.0,False,We answer Amazon support questions 7 days a we...,...,nat_says_,8.930000e+17,60186579.0,0.0,False,NaN,Aug,1.0,2017.0,Aug 1.0 2017.0
462027,'892560509639241728',Wed Aug 02 01:39:43 +0000 2017,nat_says_,60186579.0,14857.0,229.0,False,5.0,False,nothing really going on here. just talking to ...,...,AmazonHelp,8.930000e+17,85741735.0,0.0,False,NaN,Aug,2.0,2017.0,Aug 2.0 2017.0


#Task 1

In [8]:
#Step 1: Find out the users who are active in at least five listed days (i.e., created posts in at least 5 days) in Amazon_Responded_Oct05.csv and save their “user_screen_name” and “user_id_str” in the dataframe “daily_active_users” (see below). 
#Report how many active users you find.

amazon_str = amazon_data.astype(str) #changing datframe to string type before converting it to spark
spark_amazon = spark.createDataFrame(amazon_str) #converting to spark dataframe
spark_amazon = spark_amazon.select('user_screen_name','user_id_str','tweet_created_date') #selecting the three columns that would be required as per the assignment
spark_amazon.show(10) #displaying the final dataframe

+----------------+-----------+------------------+
|user_screen_name|user_id_str|tweet_created_date|
+----------------+-----------+------------------+
|             nan|        nan|               nan|
|     SeanEPanjab|143515471.0|    Nov 1.0 2016.0|
|      AmazonHelp| 85741735.0|    Nov 1.0 2016.0|
|     SeanEPanjab|143515471.0|    Nov 1.0 2016.0|
|     SeanEPanjab|143515471.0|    Nov 1.0 2016.0|
|      AmazonHelp| 85741735.0|    Nov 1.0 2016.0|
|      AmazonHelp| 85741735.0|    Nov 1.0 2016.0|
|     SeanEPanjab|143515471.0|    Nov 1.0 2016.0|
|     SeanEPanjab|143515471.0|    Nov 1.0 2016.0|
|      AmazonHelp| 85741735.0|    Nov 1.0 2016.0|
+----------------+-----------+------------------+
only showing top 10 rows



In [9]:
spark_amazon = spark_amazon.dropDuplicates() #dropping duplicates or else they would be included in the daily active users count

In [10]:
daily_active_users = spark_amazon.groupBy('user_screen_name','user_id_str').count() #taking the count of all users

In [11]:
daily_active_users = daily_active_users.filter("count>=5") #filtering out users who are active on 5 days atleast
daily_active_users = daily_active_users.select('user_screen_name','user_id_str') #selecting the two columns required to match the format as per assignment 

In [12]:
from pyspark.sql.functions import *
daily_active_users = daily_active_users.withColumn("user_id_str", regexp_replace('user_id_str', '\..*$', '')) #removing decimal point from 'user_id_str' to match the format as per assignment 

In [13]:
daily_active_users.show(10) #displaying the final output of task1

+----------------+-----------+
|user_screen_name|user_id_str|
+----------------+-----------+
|          txomon|  125675649|
|    RobynCobbold|   21041619|
|   mehulsthakker|  187257670|
|        trallyus|   11702402|
|        NOtoGMOs|   49989431|
|    matthewD2013| 1300002864|
|    ElliottBros1| 2317805748|
|      Avisingh18|  103913560|
|         Tillums|  206642342|
|       sanjanind|   66622722|
+----------------+-----------+
only showing top 10 rows



In [14]:
daily_active_users.count() #taking the count of daily active users i.e., 593

593

#Task2

In [15]:
#Step 2: A company would like to conduct an A/B test on Twitter. The experiment.txt file includes the user_id_str they selected as potential experiment targets. Please create a dataframe “experiment_user” to document the selected user id and whether they are active users (join the dataframe from step 1). 
spark_experiment = spark.read.text('C:/Users/Sachin Chavan/Downloads/experiment.txt') #importing the experiment.txt dataset as a spark dataframe
spark_experiment.show(10) #displaying the imported dataframe

+----------+
|     value|
+----------+
| 143515471|
|  85741735|
|  71457972|
|2908108256|
| 106799492|
|  59156981|
| 902137872|
| 110354554|
|  97424433|
|  62364020|
+----------+
only showing top 10 rows



In [16]:
spark_experiment = spark_experiment.withColumnRenamed("value", "user_id_exp") #assigning a logical column name
spark_experiment.show(10) #displaying the dataframe

+-----------+
|user_id_exp|
+-----------+
|  143515471|
|   85741735|
|   71457972|
| 2908108256|
|  106799492|
|   59156981|
|  902137872|
|  110354554|
|   97424433|
|   62364020|
+-----------+
only showing top 10 rows



In [17]:
#In order to join the experiment dataframe with daily active users dataframe, we use a left join as we want all the rows of experiment dataframe to be mapped to the daily active users dataframe
experiment_user = spark_experiment.join(daily_active_users,daily_active_users.user_id_str == spark_experiment.user_id_exp,"left") 

In [18]:
experiment_user.show(10) #displaying the joined dataframe

+-----------+----------------+-----------+
|user_id_exp|user_screen_name|user_id_str|
+-----------+----------------+-----------+
|  106799492|            null|       null|
|   62364020|            null|       null|
| 2908108256|            null|       null|
|  143515471|            null|       null|
|   97424433|            null|       null|
| 2706101936|            null|       null|
|  110354554| praveen_pandey_|  110354554|
|   59156981|            null|       null|
|   71457972|            null|       null|
|   85741735|      AmazonHelp|   85741735|
+-----------+----------------+-----------+
only showing top 10 rows



In [19]:
from pyspark.sql import functions as f
experiment_user = experiment_user.withColumn('active_users', f.when(f.col('user_id_exp') == f.col('user_id_str'), "Yes").otherwise("No")) #assigning the users with mapped user ids as "Yes" and the rest as "No"

In [20]:
experiment_user.show() #displaying the dataframe with the new active users column

+-----------+----------------+-----------+------------+
|user_id_exp|user_screen_name|user_id_str|active_users|
+-----------+----------------+-----------+------------+
|  106799492|            null|       null|          No|
| 2610379644|            null|       null|          No|
|   62364020|            null|       null|          No|
|  123138418|            null|       null|          No|
| 2908108256|            null|       null|          No|
|  143515471|            null|       null|          No|
|   97424433|            null|       null|          No|
| 2706101936|            null|       null|          No|
| 1510968974|            null|       null|          No|
|  110354554| praveen_pandey_|  110354554|         Yes|
| 2502172122|            null|       null|          No|
|  243716471|            null|       null|          No|
|   59156981|            null|       null|          No|
|   71457972|            null|       null|          No|
|  257376764|            null|       null|      

In [21]:
experiment_user = experiment_user.select('user_id_exp','active_users') #selecting the two columns to match the format required as per the assignment
experiment_user.show() #displaying the final dataframe

+-----------+------------+
|user_id_exp|active_users|
+-----------+------------+
|  106799492|          No|
| 2610379644|          No|
|   62364020|          No|
|  123138418|          No|
| 2908108256|          No|
|  143515471|          No|
|   97424433|          No|
| 2706101936|          No|
| 1510968974|          No|
|  110354554|         Yes|
| 2502172122|          No|
|  243716471|          No|
|   59156981|          No|
|   71457972|          No|
|  257376764|          No|
|    5654472|          No|
|   85741735|         Yes|
|  370711133|          No|
|  145579921|          No|
|  902137872|         Yes|
+-----------+------------+
only showing top 20 rows



In [22]:
exp_user_count = experiment_user.groupBy("active_users").count() #getting the count of active vs inactive users in a new dataframe

In [23]:
exp_user_count.show() #displaying the count

+------------+-----+
|active_users|count|
+------------+-----+
|          No| 4885|
|         Yes|  115|
+------------+-----+



In [24]:
print(exp_user_count.columns) #re-confirming the column names of our dataframe before percentage calculation

['active_users', 'count']


In [25]:
from pyspark.sql.window import Window
df_percent = exp_user_count.withColumn('count_percent',f.col('count')/f.sum('count').over(Window.partitionBy())*100) #calculating the percentage of active and inactive users
df_percent.show() #displaying the percentages, the percent of active users is 2.3%

+------------+-----+-------------+
|active_users|count|count_percent|
+------------+-----+-------------+
|          No| 4885|         97.7|
|         Yes|  115|          2.3|
+------------+-----+-------------+



#Task 3

In [26]:
#Please help them fill in the remaining columns by joining the dataframes you got from step 1&2 together and save the result in a dataframe “final_experiment”. (Note: For inactive users that cannot be found in “daily_active_users”, you can leave their “user_screen_name” blank or fill with “Not found”)

In [29]:
final_experiment = spark.read.csv('C:/Users/Sachin Chavan/Downloads/final_experiment.csv', header = True) #importing the final_experiment dataset as a spark dataframe
final_experiment.show() #displaying the datframe

+-----------+----+--------------+----------------+
|user_id_str|info|whether_active|user_screen_name|
+-----------+----+--------------+----------------+
|   62364020|   F|          null|            null|
| 2706101936|   M|          null|            null|
|    5654472|   F|          null|            null|
|  145579921|   F|          null|            null|
| 2502172122|   M|          null|            null|
|  243716471|   F|          null|            null|
| 2610379644|   M|          null|            null|
|  123138418|   M|          null|            null|
|  257376764|   F|          null|            null|
|  269145593|   M|          null|            null|
|  370711133|   F|          null|            null|
| 1510968974|   F|          null|            null|
| 3526380922|   M|          null|            null|
|  163413904|   F|          null|            null|
|   16980347|   M|          null|            null|
| 1209614366|   M|          null|            null|
|  447433286|   F|          nul

In [30]:
#The first join between daily active users and experiment_users dataframe would be left join to capture all active and inactive users
join1 = experiment_user.join(daily_active_users, daily_active_users.user_id_str == experiment_user.user_id_exp, how = 'left')
join1.show() #displaying the joined dataframe

+-----------+------------+----------------+-----------+
|user_id_exp|active_users|user_screen_name|user_id_str|
+-----------+------------+----------------+-----------+
|  106799492|          No|            null|       null|
| 2610379644|          No|            null|       null|
|   62364020|          No|            null|       null|
|  123138418|          No|            null|       null|
| 2908108256|          No|            null|       null|
|  143515471|          No|            null|       null|
|   97424433|          No|            null|       null|
| 2706101936|          No|            null|       null|
| 1510968974|          No|            null|       null|
|  110354554|         Yes| praveen_pandey_|  110354554|
| 2502172122|          No|            null|       null|
|  243716471|          No|            null|       null|
|   59156981|          No|            null|       null|
|   71457972|          No|            null|       null|
|  257376764|          No|            null|     

In [31]:
join1 = join1.drop('user_id_str') #dropping the user_id_str here itself as it will pose a duplication issue post the final join
join1.show(5) #displaying the dataframe post the column drop

+-----------+------------+----------------+
|user_id_exp|active_users|user_screen_name|
+-----------+------------+----------------+
|  106799492|          No|            null|
| 2908108256|          No|            null|
|  143515471|          No|            null|
|   59156981|          No|            null|
|   71457972|          No|            null|
+-----------+------------+----------------+
only showing top 5 rows



In [32]:
final_experiment = final_experiment.drop('user_screen_name') #As join1 already has user_screen_name, dropping this column from final_experiment to avoid duplication post the final join

In [33]:
#The final join between final_experiment and join1 would also be left join to capture the include the users in the revised experiment list
join2 = final_experiment.join(join1, final_experiment.user_id_str == join1.user_id_exp, how = 'left')
join2.show() #displaying the joined dataframe

+-----------+----+--------------+-----------+------------+----------------+
|user_id_str|info|whether_active|user_id_exp|active_users|user_screen_name|
+-----------+----+--------------+-----------+------------+----------------+
| 1112166661|   F|          null| 1112166661|          No|            null|
| 1209614366|   M|          null| 1209614366|          No|            null|
|  123138418|   M|          null|  123138418|          No|            null|
|  145579921|   F|          null|  145579921|          No|            null|
| 1510968974|   F|          null| 1510968974|          No|            null|
|  163413904|   F|          null|  163413904|          No|            null|
|   16980347|   M|          null|   16980347|          No|            null|
| 1970607968|   M|          null| 1970607968|          No|            null|
|  243716471|   F|          null|  243716471|          No|            null|
| 2502172122|   M|          null| 2502172122|          No|            null|
|  257376764

In [34]:
join2 = join2.drop('whether_active', 'user_id_exp') #dropping the column to match the format as per the assigned task
join2.show(5) #displaying the dataframe

+-----------+----+------------+----------------+
|user_id_str|info|active_users|user_screen_name|
+-----------+----+------------+----------------+
|  145579921|   F|          No|            null|
|  243716471|   F|          No|            null|
| 2502172122|   M|          No|            null|
| 2706101936|   M|          No|            null|
|    5654472|   F|          No|            null|
+-----------+----+------------+----------------+
only showing top 5 rows



In [35]:
join2 = join2.withColumnRenamed("active_users", "whether_active") #renaming the column to match the format as per the assigned task
join2.show() #displaying the final joined dataframe

+-----------+----+--------------+----------------+
|user_id_str|info|whether_active|user_screen_name|
+-----------+----+--------------+----------------+
| 1112166661|   F|            No|            null|
| 1209614366|   M|            No|            null|
|  123138418|   M|            No|            null|
|  145579921|   F|            No|            null|
| 1510968974|   F|            No|            null|
|  163413904|   F|            No|            null|
|   16980347|   M|            No|            null|
| 1970607968|   M|            No|            null|
|  243716471|   F|            No|            null|
| 2502172122|   M|            No|            null|
|  257376764|   F|            No|            null|
| 2610379644|   M|            No|            null|
|  269145593|   M|            No|            null|
| 2706101936|   M|            No|            null|
| 3285473358|   F|           Yes|    iwritegarima|
| 3526380922|   M|            No|            null|
|  370711133|   F|            N

In [36]:
final_experiment = join2 #assigning the new joined dataframe to final_experiment
final_experiment.show() #displaying the final_experiment dataframe

+-----------+----+--------------+----------------+
|user_id_str|info|whether_active|user_screen_name|
+-----------+----+--------------+----------------+
| 1112166661|   F|            No|            null|
| 1209614366|   M|            No|            null|
|  123138418|   M|            No|            null|
|  145579921|   F|            No|            null|
| 1510968974|   F|            No|            null|
|  163413904|   F|            No|            null|
|   16980347|   M|            No|            null|
| 1970607968|   M|            No|            null|
|  243716471|   F|            No|            null|
| 2502172122|   M|            No|            null|
|  257376764|   F|            No|            null|
| 2610379644|   M|            No|            null|
|  269145593|   M|            No|            null|
| 2706101936|   M|            No|            null|
| 3285473358|   F|           Yes|    iwritegarima|
| 3526380922|   M|            No|            null|
|  370711133|   F|            N

In [37]:
final_experiment.groupBy("whether_active").count().show() #taking the count to ensure all users (4500) were included in the final_experiment dataframe

+--------------+-----+
|whether_active|count|
+--------------+-----+
|          null|  262|
|            No| 4142|
|           Yes|   96|
+--------------+-----+



In [38]:
#pushing the daily_active_users dataframe to csv
daily_active_users.toPandas().to_csv(('C:/Users/Sachin Chavan/Downloads/daily_active_users_sachin.csv'), index = True, header=True)

In [39]:
#pushing the experiment_user dataframe to csv
experiment_user.toPandas().to_csv(('C:/Users/Sachin Chavan/Downloads/experiment_user_sachin.csv'), index = True, header=True)

In [40]:
#pushing the final_experiment dataframe to csv
final_experiment.toPandas().to_csv(('C:/Users/Sachin Chavan/Downloads/final_experiment_sachin.csv'), index = True, header=True)